<h1>Korean Text classification with KoBERT</h1>
This is the final project for the class of Deep learning showing understanding of the deep learning concepts.

## **Summary of the project**

In Korea, even though there are many research conducted being conducted on voice phishing, it remains a real case problem that technologye suchas artificial inteligence can tackle. Through a previous project conducted, I createtd a dataset containing phone call convesation transcipts and general conversation text daata. This voice phishing dataset has two differents class which are **voice phishing** (represented as "1") and **non-voice phishing** (represented as "0"). 

Using this dataset with state-of-the-art (SOTA) pre-trained word embedding [KoBERT](https://github.com/SKTBrain/KoBERT), we will perform NLP task such as text classification to build binary classification models.

## **Aim of the project**
In this project, we aim to build binary classification models capable to determine whether the inputted Korean conversation text is voice phishing ("1") or non-voice phishing ("0") related text.

The API used are Tensorflow for BERT model and Pytorch fror KoBERT model.

## **Desired outputs of the project**
From the trained models, we expect to achieve great classification performance on this voice phishing dataset such as the model tells us if a conversation is harmful or not harmful.
At the end of this project, we will look at the accuracy of the model on the test set.

# Training the binary classificatioon model with KoBERT

## Installing the common needed packages

In [1]:
# dowload and install KoBERT as a python package
# this commande will install the requirted package at the same time
  # gluonnlp >= 0.6.0
  # mxnet >= 1.4.0
  # onnxruntime >= 0.3.0
  # sentencepiece >= 0.1.6
  # torch >= 1.7.0
  # transformers >= 4.8.1

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ojy1mzfd
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ojy1mzfd
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [2]:
# !pip list

## Import all the needed libraries

In [3]:
## importing the required packages
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

from sklearn.model_selection import train_test_split

ImportError: /home/phenomx/anaconda3/envs/dltest/lib/python3.10/site-packages/gluonnlp/data/fast_bert_tokenizer.cpython-310-x86_64-linux-gnu.so: undefined symbol: _PyGen_Send

In [ ]:
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential

In [ ]:
## importing KoBERT functions
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
## import transformers functions
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
## Configure the GPU  device
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

## Importing the dataset

In [ ]:
"""
Since we are using Colab, we will provide a test to check if environment is 
colab or not so that the data can also be imported in case this jupyter file is 
ran on local machine and not on colab
"""

if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  ## mount the google drive
  from google.colab import drive
  drive.mount('drive')
  # move to the directory where dataset is saved
  %cd drive/My\ Drive/Colab\ Notebooks/
else:
  print('Not running on CoLab')

In [ ]:
# import the dataset
# dataset = pd.read_csv('KorCCViD_v1.3_fullcleansed.csv').sample(frac=1.0)
dataset = pd.read_csv('KorCCVi_v2.csv')
dataset

In [ ]:
dataset.info()

In [ ]:
dataset['label'].value_counts()

# Dataset cleaning and purification

In [ ]:
# drop the colum we don't need
dataset.drop(['confidence', 'id'], axis=1, inplace=True)
dataset

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
dataset.hist()
plt.show()

## Dataset cleaning and purification

In [ ]:
# function to perform the cleaning parts
def apply_replacement(src_df, replace_func):
    ret_df = src_df
    ret_df['transcript'] = ret_df['transcript'].apply(lambda x: replace_func(x))
    return ret_df

In [ ]:
%%time
# remove the unwanted word and characters from the dataset
example_word_replace_list = {'o/': '',
                             'b/': '',
                             'n/': '',
                             '\n': ' ',
                             'name': '',
                             'laughing': '',
                             'clearing': '',
                             'singing': '',
                             'applauding': ''}

def word_replace(x):
    for i in example_word_replace_list:
        x = x.replace(i, example_word_replace_list[i])
    return x

dataset = apply_replacement(dataset, word_replace)

In [ ]:
dataset

In [ ]:
%%time
# remove the special character from the transcripts
import re

# dataset_clear_text = list(dataset['Transcript'])

# dataset_clear_text2 = []

# for text in dataset_clear_text:
#   temp = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)
#   dataset_clear_text2.append(temp)
# dataset['Transcript'] = dataset_clear_text2

def remove_special_sysmbols(sentence): 
    sentence = re.sub(r"[-~=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]", '', sentence)
    return sentence

dataset = apply_replacement(dataset, remove_special_sysmbols)

In [ ]:
dataset

In [ ]:
# remove x and O from the transcripts
%%time
def replace_x_o(sentence):
    
    # sentence = re.sub(r"(o{2,})|(O{2,})|(\ㅇ{2,})|(0{2,})|(x{2,})", '' , sentence)
    # sentence = re.sub(r"x{2,}", '' , sentence)
    # sentence = re.sub(r"0{2,}", '' , sentence)
    # sentence = re.sub(r"ㅇ{2,}", '' , sentence)
    # sentence = re.sub(r"O{2,}", '' , sentence)
    # sentence = re.sub(r"o{2,}", '' , sentence)
    sentence = re.sub(r"(o|O|\ㅇ|0|x){2,}", '' , sentence)
    return sentence
    
dataset = apply_replacement(dataset, replace_x_o)

In [ ]:
dataset

In [ ]:
%%time
# remove the unwanted word and characters from the transcripts
example_word_replace_list = {'\n' : ' '}

def word_replace(x):
    for i in example_word_replace_list:
        x = x.replace(i, example_word_replace_list[i])
    return x

dataset = apply_replacement(dataset, word_replace)

In [ ]:
dataset

In [ ]:
print(dataset.groupby('label').size().reset_index(name='count'))

In [ ]:
# save the cleansed dataset of the raw dataset has been used
dataset.to_csv("KorCCVi_v2_halfcleansed.csv")

## Morphology Installation

### Installation of MeCab analyzer
#### To use Mecab, you need to download both mecab-ko and mecab-ko-dic.

#### Method 2: Installation of Mecab-ko-for-GoogleColab

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab

In [ ]:
# !cd Colab Notebooks
!pwd
!ls -l

In [ ]:
%%time
# !bash install_mecab-ko_on_colab190912.sh
!bash install_mecab-ko_on_colab_light_220429.sh

In [ ]:
cd ..

##### Test of MeCab

In [ ]:
# !pip install konlpy

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()
text = '그중에서 제일 기 좋았던 거는 스페인이랑 포르투갈이었거든요.'

In [ ]:
my_morphs = mecab.morphs(text)
print(my_morphs)

In [ ]:
mecab.pos(text)

In [ ]:
tokenizer = Mecab()
tokenizer.morphs(text)

In [ ]:
%%time
dataset_doc = [(mecab.morphs(x),y) for x,y in zip(dataset['transcript'],dataset['label'])]

In [ ]:
dataset_doc

In [ ]:
## remove the stop word
stopwords = ["을", "를", "이", "가", "ㅡ", "은", "는", "XXX", "xxx", "어요", "아니", "입니다", "에서", "니까", "으로",
             "근데", "습니다", "습니까", "저희", "합니다", "하고", "싶어요", "있는", "있습니다", "싶습니다", "그냥",
             "고요", "에요", "예요", "으시", "그래서"]

# function to remove the stop word from the dataframe
def get_model_input(_words):
    global stopwords
    _words = [x for x in _words if x[0] not in stopwords]
    _words = [x for x in _words if x[:-1] not in stopwords]

    for i in range(len(_words)-1):
        yield _words[i]

In [ ]:
%%time
X_dataset, Y_dataset = [], []
for lwords in dataset_doc:
    Y_dataset.append(lwords[1])
    
    temp = []
    for x in get_model_input(lwords[0]):
        if len(x) != 1:
            temp.append("{}".format(x))
    X_dataset.append(" ".join(temp))

In [ ]:
# Rebuild the full cleansed train and test set in dataframe for future use
train_dic = {'transcript':X_dataset, 'label':Y_dataset}
df_train_fullCleansed = pd.DataFrame(train_dic)

In [ ]:
df_train_fullCleansed

In [ ]:
# save the cleansed dataset
df_train_fullCleansed.to_csv("KorCCVi_v2_fullcleansed.csv",index=False)

In [ ]:
ls -l

## Data transformation and splitting

In [ ]:
## transform our train set and test set into tsv file to usedd into KoBERT
# train_tsv = nlp.data.TSVDataset('KorCCViD_v1.3_fullcleansed.csv')
# train_tsv = nlp.data.TSVDataset('KorCCViD_v1.3_fullcleansed.csv')

dataset_tsv = []
for text, label in zip(df_train_fullCleansed['transcript'], df_train_fullCleansed['label']):
    data = []
    data.append(text)
    data.append(str(label))

    dataset_tsv.append(data)

In [ ]:
dataset_tsv[:5]

In [ ]:
# Split the data into train set and test set

train_set, test_set = train_test_split(dataset_tsv, 
                               test_size=0.3, 
                               random_state=42, 
                               shuffle=True)
print(f"Numbers of train instances by class: {len(train_set)}")
print(f"Numbers of test instances by class: {len(test_set)}")

## Prepare the data as input for the KoBERT model
According tot he documentation the class BERTDataset is to be used to perform in the background the following tasks.
- Tokenization
- Numericalization (encoding string to integer)
- Padding
- etc



In [ ]:
# Definition of BERTDataset class (mandatory)
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting the hyperparameters
max_len = 64 # The maximum sequence length that this model might ever be used with. 
             # Typically set this to something large just in case (e.g., 512 or 1024 or 2048).
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10   # only parameter changed from 5 to 10 compared to the documentation
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5  # 4e-5

In [ ]:
# Perform the prearation task of the data using class defined above
bertmodel, vocab = get_pytorch_kobert_model() # calling the bert model and the vocabulary

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

train_set = BERTDataset(train_set, 0, 1, tok, max_len, True, False)
test_set = BERTDataset(test_set, 0, 1, tok, max_len, True, False)

In [ ]:
tokenizer

In [ ]:
tok

In [ ]:
# verifying the transformation
train_set[1]

In [ ]:
# creating torch-type datasets
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)

## Creation of the KoBERT learing model

In [ ]:
# This class is from the github repository and the documentation
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   # since we are in binary classification we set the value 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
import torch
# torch.cuda.empty_cache()

In [ ]:
# creation of the model
model = BERTClassifier(bertmodel,  dr_rate=0.4).to(device)

In [ ]:
# print(type(model))

In [ ]:
%%time
print(model)

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The KoBERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
# optimizer_grouped_parameters

In [ ]:
# configurtation f the optimizer and loss function
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
scheduler

In [ ]:
# define the function to compute the accury of the model
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# # Function to calculate the accuracy of our predictions vs labels
# def flat_accuracy(preds, labels):
#     pred_flat = np.argmax(preds, axis=1).flatten()
#     labels_flat = labels.flatten()
#     return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# !pip install seqeval==0.0.5
# from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

# def f1_pre_rec(labels, preds):
#     return {
#         "precision": precision_score(labels, preds, suffix=True),
#         "recall": recall_score(labels, preds, suffix=True),
#         "f1": f1_score(labels, preds, suffix=True)
#     }

In [ ]:
# out.cpu().detach().numpy() 

In [ ]:
# label.cpu().detach().numpy() 

In [ ]:
def get_metrics(pred, label, threshold=0.5):
    pred = (pred > threshold).astype('float32')
    tp = ((pred == 1) & (label == 1)).sum()
    fp = ((pred == 1) & (label == 0)).sum()
    fn = ((pred == 0) & (label == 1)).sum()
    
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * recall * precision / (precision + recall)
    
    return {
        'recall': recall,
        'precision': precision,
        'f1': f1
    }

## Training the KoBERT model


In [ ]:
%%time
from time import time
from timeit import default_timer as timer
from sklearn.metrics import *
# Training code from the github library
start_time = time()

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    train_pres = 0.0
    test_pres = 0.0

    # Training of the model with the train set
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        #train_pres = precision_score(label, out, average='micro')
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    # evaluation of the model train on the test set
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        labe2 = label.cpu()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} val acc {}".format(e+1, test_acc / (batch_id+1)))
    # print('ACCURACY 2 = ', accuracy_score(out, label))

run_time = time() - start_time

In [ ]:
run_time
#224.96386766433716
#0.9947

In [ ]:
# # graph of the training
# pd.DataFrame(history.history).plot(figsize=(8, 5))
# plt.grid(True)
# plt.legend()
# plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
# plt.xlabel('epochs')
# plt.show()
# # plt.savefig('bilstm_all_metrics.png')

In [ ]:
# # accuracy: (tp + tn) / (p + n)
# accuracy = accuracy_score(Y_TEST, yhat_classes)
# print('Testing Accuracy: %f' % accuracy)

# precision tp / (tp + fp)
#precision = precision_score(Y_TEST, yhat_classes,average='weighted')
#print('Testing Precision: %f' % precision)

# # recall: tp / (tp + fn)
# recall = recall_score(Y_TEST, yhat_classes,average='weighted')
# print('Testing Recall: %f' % recall)
# # f1: 2 tp / (2 tp + fp + fn)
# f1 = f1_score(Y_TEST, yhat_classes,average='weighted')
# print('Testing F1 score: %f' % f1)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

# def accuracy_per_class(preds, labels):
#     label_dict_inverse = {v: k for k, v in label_dict.items()}
    
#     preds_flat = np.argmax(preds, axis=1).flatten()
#     labels_flat = labels.flatten()

#     for label in np.unique(labels_flat):
#         y_preds = preds_flat[labels_flat==label]
#         y_true = labels_flat[labels_flat==label]
#         print(f'Class: {label_dict_inverse[label]}')
#         print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
preds = []
labels = []
test_acc = 0.0
# evaluation of the model train on the test set
model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    labe2 = label.cpu()
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)

    pred = out.detach()
    pred = F.softmax(pred)
    pred = pred[:, 1].cpu().numpy().tolist()
    preds += pred
    labels += label.cpu().numpy().tolist()
    
preds = np.array(preds)
labels = np.array(labels)
metrics = get_metrics(preds, labels)
print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
# print('ACCURACY 2 = ', accuracy_score(out, label))
print('Metrics: ', metrics)

<h2>Model Training result</h2>

From the previous training result, we can see that our KoBERT binary classification model reached **99.68%** of accuracy on the test set and **100**%  of accuracy on the train set.

# [Model state_dict](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [ ]:
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

# Saving & Loading Model for Inference

In [ ]:
!ls -l

In [ ]:
#save
PATH = './weights/'

torch.save(model.state_dict(), PATH + 'KoBERT_vishing_weights.pt') # Save state_dict
torch.save(model, PATH + 'KoBERT_vishing_model.pt')  # Save entire Model

In [ ]:
# Store multiple values, save normal scalar values such as epoch and loss values to store progress during learning
#can be very heavy
# torch.save({
#     'model': model.state_dict(),
#     'optimizer': optimizer.state_dict()
# }, PATH + 'all.tar')

In [ ]:
#Load 
# model = BERTClassifier()
model.load_state_dict(torch.load(PATH + 'KoBERT_vishing_weights.pt'))
model.eval()

# [Apply the dynamic quantization](https://pytorch.org/tutorials/intermediate/dynamic_quantization_bert_tutorial.html#apply-the-dynamic-quantization)

In [ ]:
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
print(quantized_model)